In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

In [10]:
data = pd.read_csv('C:/Users/annar/Downloads/sample.csv')

In [26]:
data['Date'] = pd.to_datetime(data.iloc[:,5])

# create a new column for the day of the year (1-365)
data['day_of_year'] = data.iloc[:,5].dt.dayofyear

data['year'] = data.iloc[:,5].dt.year

for i in range (32):
    data['AVG'] = (data.iloc[:,6] + data.iloc[:,7])/2

In [27]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

train_X = train[['day_of_year', 'year']]
train_y = train['AVG']

test_X = test[['day_of_year', 'year']]
test_y = test['AVG']


In [41]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# create the model
model = Sequential()

# add the LSTM layer
model.add(LSTM(50, input_shape=(2, 1)))

# add the output layer
model.add(Dense(1))

# compile the model
model.compile(loss='mean_squared_error', optimizer='adam')



In [45]:
train_X = train_X.values.reshape((train_X.shape[0], 2, 1))
test_X = test_X.values.reshape((test_X.shape[0], 2, 1))

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [46]:
model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2)

# create a new dataframe for the future dates
future_dates = pd.date_range(start='2010-03-01', end='2010-03-31')
future_data = pd.DataFrame({'date': future_dates})

# create the day of year and year columns for the future dates
future_data['day_of_year'] = future_data['date'].dt.dayofyear
future_data['year'] = future_data['date'].dt.year

# make predictions using the model
future_X = future_data[['day_of_year', 'year']]
future_X = future_X.values.reshape((future_X.shape[0], 2, 1))
future_y = model.predict(future_X)

# add the predicted temperatures to the dataframe
future_data['temperature'] = future_y

# print the predicted temperatures
print(future_data['temperature'])


Epoch 1/50
1/1 - 0s - loss: 26569.2793 - val_loss: 29821.1309 - 66ms/epoch - 66ms/step
Epoch 2/50
1/1 - 0s - loss: 26565.9043 - val_loss: 29817.6875 - 33ms/epoch - 33ms/step
Epoch 3/50
1/1 - 0s - loss: 26562.4844 - val_loss: 29814.0156 - 25ms/epoch - 25ms/step
Epoch 4/50
1/1 - 0s - loss: 26558.8301 - val_loss: 29809.4355 - 30ms/epoch - 30ms/step
Epoch 5/50
1/1 - 0s - loss: 26554.2754 - val_loss: 29803.4102 - 32ms/epoch - 32ms/step
Epoch 6/50
1/1 - 0s - loss: 26548.2832 - val_loss: 29798.1973 - 32ms/epoch - 32ms/step
Epoch 7/50
1/1 - 0s - loss: 26543.0957 - val_loss: 29794.2637 - 32ms/epoch - 32ms/step
Epoch 8/50
1/1 - 0s - loss: 26539.1875 - val_loss: 29790.5332 - 32ms/epoch - 32ms/step
Epoch 9/50
1/1 - 0s - loss: 26535.4785 - val_loss: 29786.8066 - 33ms/epoch - 33ms/step
Epoch 10/50
1/1 - 0s - loss: 26531.7715 - val_loss: 29783.0586 - 32ms/epoch - 32ms/step
Epoch 11/50
1/1 - 0s - loss: 26528.0449 - val_loss: 29779.2910 - 32ms/epoch - 32ms/step
Epoch 12/50
1/1 - 0s - loss: 26524.2988 -

In [33]:
Pkl_file = 'pickle_model'
with open(Pkl_file, 'wb') as file:
    pickle.dump(model,file)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-05 09:46:17         1808
metadata.json                                  2023-03-05 09:46:17           64
variables.h5                                   2023-03-05 09:46:17       143648


In [34]:
Pkl_file = 'pickle_model'
with open(Pkl_file, 'rb') as file:
    predictor=pickle.load(file)

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-05 09:46:16         1808
metadata.json                                  2023-03-05 09:46:16           64
variables.h5                                   2023-03-05 09:46:16       143648
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


In [35]:
def preprocess_data(data):
    return np.reshape(data, (data.shape[0], data.shape[1], 1))

# Define a function to make predictions based on user input data
def predict_weather(data):
    data = preprocess_data(data)
    return predictor.predict(data)

In [37]:
city = input("Enter a city name: ")
d = data[city]
prediction = predict_weather(d)
print('The predicted weather for the next month is:', prediction[0])

KeyError: 'petersburg'